# Child Tax Credit

The Child Tax Credit (CTC) is a partially refundable tax credit for filers with children that phases in and out with income.

In [1]:
from policyengine_us.tools.documentation import (
    variable_summary,
    variation_chart,
)

variable_summary("ctc")

,Value
Metadata,
Name,ctc
Label,Child Tax Credit
Entity,tax_unit
Definition period,year
Has a formula,True
Value type,float
Unit,currency-USD
Documentation,Total value of the non-refundable and refundab...
Reference,[https://www.law.cornell.edu/uscode/text/26/24#a]


In [2]:
variation_chart("ctc")